In [ ]:
#packages inlezen
from netCDF4 import Dataset, chartostring
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, time
import hkvsobekpy as hkv
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from tqdm import tqdm
%matplotlib inline

In [ ]:
#paden opgeven naar modellen
#alleen de modelmapnaam aanpassen

#DHydro_Model_his= os.path.abspath('../03_Model/06_1D2DmodelGoorloop.dsproj_data/FlowFM/output/FlowFM_his.nc') #Goorloop
#DHydro_Model_map = os.path.abspath('../03_Model/06_1D2DmodelGoorloop.dsproj_data/FlowFM/output/FlowFM_map.nc') #Goorloop
DHydro_Model_his= os.path.abspath('../03_Model/Hwet2Dtestrun2.dsproj_data/FlowFM/output/FlowFM_his.nc') #Hertogswetering
DHydro_Model_map = os.path.abspath('../03_Model/Hwet2Dtestrun2.dsproj_data/FlowFM/output/FlowFM_map.nc') #Hertogswetering

opslaan = True

In [ ]:
#his en map bestand uit D-Hydro uitlezen
his = Dataset(DHydro_Model_his)
network = Dataset(DHydro_Model_map)

## Data uitlezen uit D-Hydro

In [ ]:
#D-Hydro 1D
node_id = chartostring(network['mesh1d_node_id'][:])
node_id = [nodeid.rstrip() for nodeid in node_id]

In [ ]:

ref_time = ' '.join(his['time'].units.split()[2:4])
print('reference time D-Hydro:',ref_time)
tijd = his['time'][:]

ref = datetime.datetime.strptime(ref_time, "%Y-%m-%d %H:%M:%S")

index_his = [ref + datetime.timedelta(seconds=each) for each in tijd]

## Waterbalans

In [ ]:
#waterbalans termen uitlezen uit D-Hydro his bestand
in_bound = pd.DataFrame(data = his['water_balance_boundaries_in'][:], index=index_his,columns=['Boundaries in'])
out_bound = pd.DataFrame(data = his['water_balance_boundaries_out'][:], index=index_his, columns=['Boundaries out'])
total_bound = pd.DataFrame(data = his['water_balance_boundaries_total'][:], index=index_his)
storage = pd.DataFrame(data = his['water_balance_storage'][:], index=index_his, columns=['Storage'])
lateral = pd.DataFrame(data = his['water_balance_laterals_total'][:], index=index_his, columns=['Lateral disch. tot.'])
net_1D2D_in = pd.DataFrame(data = his['water_balance_exchange_with_1D_total'][:], index=index_his, columns=['Net 1D2D in'])
#sink = pd.DataFrame(data = his['water_balance_source_sink'][:], index=index_his)
totaal = pd.DataFrame(data = his['water_balance_total_volume'][:], index=index_his, columns=['Volume water system']) 
#time = pd.DataFrame(data = his['water_balance_total_volume'][:], index=index_his) 
in2D = pd.DataFrame(data = his['water_balance_exchange_with_1D_total'][:], index=index_his, columns=['in2D'])

#data = [his['water_balance_boundaries_in'][:], his['water_balance_boundaries_out'][:]]
data = [in_bound,out_bound,lateral,storage, totaal]
dhydro_balans = pd.concat(data, axis=1)

In [ ]:
#typ hier plotly om interactieve grafieken te genereren, anders worden er statische afbeeldingen genereert via matplotlib (makkelijker te exporteren)
plot = 'plotly'

#waterstanden measurement stations
for i in ['Boundaries in', 'Boundaries out', 'Lateral disch. tot.','Storage', 'Volume water system']:
    if plot == 'plotly':
        fig = px.line(dhydro_balans, x = dhydro_balans.index,y = [dhydro_balans.loc[:,i]], title=i,
                      labels={"value": "Volume [m3]",  "index": "Datum"},
                     template='plotly') #template aannpassen, opties zijn:     
        #['ggplot2', 'seaborn', 'simple_white', 'plotly',
        #'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
        #'ygridoff', 'gridon', 'none']

        #verschillende opties voor aan en uitzetten van hoverinfo en opmaak ervan
        fig.update_traces(mode="lines", hovertemplate=None)
        fig.update_layout(hovermode="x unified")
        
        #wegschrijven van figuren
        if opslaan == True:
            pio.write_image(fig,'../02_Validatie/Hertogswetering/Waterbalans/svg/balans_'+ i +'.svg', engine='orca')
            fig.write_html('../02_Validatie/Hertogswetering/Waterbalans/html/balans_'+ i +'.html', include_plotlyjs="cdn", full_html=False)

        fig.show()
        

        
    else:
        fig,ax = plt.subplots(figsize=(20, 10))
        ax.plot(df.index,df.loc[:,locatie])
        ax.grid()
        ax.tick_params(labelsize=16)
        ax.legend(['D-Hydro'], fontsize = 16)
        ax.set_xlabel('Datum',fontsize=16)
        ax.set_ylabel('waterstand [m NAP]',fontsize=16)
        ax.set_title(locatie,fontsize=16)
        #fig.savefig(Gebied+'/meetpunt/waterstand bij meetpunt {}_V2.png'.format(locatie),bbox_inches='tight')